In [7]:
#import dependencies
import pandas as pd
from sqlalchemy import create_engine
from config import password

In [8]:
# Load in CSV population data
data = pd.read_csv("Resources/PEP_2017_PEPANNRES_with_ann.csv", encoding='latin-1')
data.head()

,GEO.id,GEO.id2,GEO.display-label,rescen42010,resbase42010,respop72010,respop72011,respop72012,respop72013,respop72014,respop72015,respop72016,respop72017
0,Id,Id2,Geography,"April 1, 2010 - Census","April 1, 2010 - Estimates Base",Population Estimate (as of July 1) - 2010,Population Estimate (as of July 1) - 2011,Population Estimate (as of July 1) - 2012,Population Estimate (as of July 1) - 2013,Population Estimate (as of July 1) - 2014,Population Estimate (as of July 1) - 2015,Population Estimate (as of July 1) - 2016,Population Estimate (as of July 1) - 2017
1,1620000US0100124,100124,"Abbeville city, Alabama",2688,2688,2684,2677,2629,2612,2595,2587,2575,2567
2,1620000US0100460,100460,"Adamsville city, Alabama",4522,4522,4516,4502,4479,4457,4437,4409,4376,4347
3,1620000US0100484,100484,"Addison town, Alabama",758,754,751,751,744,743,740,734,734,728
4,1620000US0100676,100676,"Akron town, Alabama",356,356,355,345,345,341,337,335,332,332


In [9]:
# Grab only relevant columns
data_col = data[['GEO.display-label', 'rescen42010', 'respop72017']]
data_col.head()

,GEO.display-label,rescen42010,respop72017
0,Geography,"April 1, 2010 - Census",Population Estimate (as of July 1) - 2017
1,"Abbeville city, Alabama",2688,2567
2,"Adamsville city, Alabama",4522,4347
3,"Addison town, Alabama",758,728
4,"Akron town, Alabama",356,332


In [10]:
# Rename columns
data_col.columns = ['Location', '2010_Population', '2017_Population']
data_col.head()

,Location,2010_Population,2017_Population
0,Geography,"April 1, 2010 - Census",Population Estimate (as of July 1) - 2017
1,"Abbeville city, Alabama",2688,2567
2,"Adamsville city, Alabama",4522,4347
3,"Addison town, Alabama",758,728
4,"Akron town, Alabama",356,332


In [11]:
# Delete first row
data_col_1 = data_col.drop(data_col.index[0])
data_col_1.head()


,Location,2010_Population,2017_Population
1,"Abbeville city, Alabama",2688,2567
2,"Adamsville city, Alabama",4522,4347
3,"Addison town, Alabama",758,728
4,"Akron town, Alabama",356,332
5,"Alabaster city, Alabama",30352,33202


In [12]:
# Reset index
data_col_1.reset_index(drop = True, inplace=True)
data_col_1.head()

,Location,2010_Population,2017_Population
0,"Abbeville city, Alabama",2688,2567
1,"Adamsville city, Alabama",4522,4347
2,"Addison town, Alabama",758,728
3,"Akron town, Alabama",356,332
4,"Alabaster city, Alabama",30352,33202


In [14]:
# Split City column into City aqnd State
data_split = data_col_1["Location"].str.split(" city,| town,", expand = True)
data_split.head(40)

,0,1
0,Abbeville,Alabama
1,Adamsville,Alabama
2,Addison,Alabama
3,Akron,Alabama
4,Alabaster,Alabama
5,Albertville,Alabama
6,Alexander City,Alabama
7,Aliceville,Alabama
8,Allgood,Alabama
9,Altoona,Alabama


In [16]:
# Insert City and State into data_col_1 dataframe
data_col_1['City'] = data_split[0]
data_col_1['State'] = data_split[1]
data_col_1.head()

,Location,2010_Population,2017_Population,City,State
0,"Abbeville city, Alabama",2688,2567,Abbeville,Alabama
1,"Adamsville city, Alabama",4522,4347,Adamsville,Alabama
2,"Addison town, Alabama",758,728,Addison,Alabama
3,"Akron town, Alabama",356,332,Akron,Alabama
4,"Alabaster city, Alabama",30352,33202,Alabaster,Alabama


In [17]:
# Convert Poppulation Columns to int
data_col_1["2010_Population"] = pd.to_numeric(data_col_1['2010_Population'], errors='coerce')
data_col_1["2017_Population"] = pd.to_numeric(data_col_1['2017_Population'], errors='coerce')
data_col_1.dtypes

Location            object
2010_Population    float64
2017_Population      int64
City                object
State               object
dtype: object

In [18]:
# Calculate poppulation change
data_col_1['Delta'] = data_col_1['2017_Population'] - data_col_1['2010_Population']
data_col_1.head()

,Location,2010_Population,2017_Population,City,State,Delta
0,"Abbeville city, Alabama",2688.0,2567,Abbeville,Alabama,-121.0
1,"Adamsville city, Alabama",4522.0,4347,Adamsville,Alabama,-175.0
2,"Addison town, Alabama",758.0,728,Addison,Alabama,-30.0
3,"Akron town, Alabama",356.0,332,Akron,Alabama,-24.0
4,"Alabaster city, Alabama",30352.0,33202,Alabaster,Alabama,2850.0


In [19]:
# Rearrange Columns 
data_cleaned = data_col_1[['City', 'State', '2010_Population', '2017_Population', "Delta"]]
data_cleaned.head()

,City,State,2010_Population,2017_Population,Delta
0,Abbeville,Alabama,2688.0,2567,-121.0
1,Adamsville,Alabama,4522.0,4347,-175.0
2,Addison,Alabama,758.0,728,-30.0
3,Akron,Alabama,356.0,332,-24.0
4,Alabaster,Alabama,30352.0,33202,2850.0


In [20]:
data_cleaned.dtypes

City                object
State               object
2010_Population    float64
2017_Population      int64
Delta              float64
dtype: object

In [21]:
# Add Positive/Negative Column
def neg_pos(row):
    change = row["Delta"]
    if change >= 0:
        return "Positive"
    else:
        return "Negative"

In [22]:
data_cleaned["Neg_Pos"]=data_cleaned.apply(neg_pos, axis=1)
data_cleaned.head()

,City,State,2010_Population,2017_Population,Delta,Neg_Pos
0,Abbeville,Alabama,2688.0,2567,-121.0,Negative
1,Adamsville,Alabama,4522.0,4347,-175.0,Negative
2,Addison,Alabama,758.0,728,-30.0,Negative
3,Akron,Alabama,356.0,332,-24.0,Negative
4,Alabaster,Alabama,30352.0,33202,2850.0,Positive


In [23]:
# Connect to Local SQL Database
connection = f"root:{password}@localhost/quality_db"
engine = create_engine(f"mysql://{connection}")

In [24]:
# Confirm tables
engine.table_names()

OperationalError: (MySQLdb._exceptions.OperationalError) (1045, "Access denied for user 'root'@'localhost' (using password: NO)")
(Background on this error at: http://sqlalche.me/e/e3q8)